In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./ratings.dat', sep=',', header=None, names=['uid', 'mid', 'rating', 'timestamp'])
len(df)

23181

In [7]:
def dataset_filter(ratings, min_items=5):
    """
            Only keep the data useful, which means:
                - all ratings are non-zeros
                - each user rated at least {self.min_items} items
            :param ratings: pd.DataFrame
            :param min_items: the least number of items user rated
            :return: filter_ratings: pd.DataFrame
            """

    # filter unuseful data
    ratings = ratings[ratings['rating'] > 0]

    # only keep users who rated at least {self.min_items} items
    user_count = ratings.groupby('uid').size()
    user_subset = np.in1d(ratings.uid, user_count[user_count >= min_items].index)
    filter_ratings = ratings[user_subset].reset_index(drop=True)

    del ratings

    return filter_ratings

In [14]:
df = pd.read_csv('./amazon.edges', sep=",", header=None, names=['uid', 'mid', 'rating', 'timestamp'],engine='python')

In [15]:
ratings = dataset_filter(df, 10)

In [16]:
user_id = ratings[['uid']].drop_duplicates().reindex()
user_id['userId'] = np.arange(len(user_id))
ratings = pd.merge(ratings, user_id, on=['uid'], how='left')

item_id = ratings[['mid']].drop_duplicates()
item_id['itemId'] = np.arange(len(item_id))
ratings = pd.merge(ratings, item_id, on=['mid'], how='left')

ratings = ratings[['userId', 'itemId', 'rating', 'timestamp']].sort_values(by='userId', ascending=True)

In [18]:
ratings.to_csv('./ratings.dat', sep=',', index=False, header=False)

In [20]:
len(set(ratings.itemId))

7957